# Engenharia de Atributos

In [122]:
import pandas as pd

In [123]:
merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')

C:\Users\Matheus\AppData\Local\Temp\ipykernel_20812\3999875462.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')


Convertendo as variáveis para os seus respectivos tipos corretos 

In [124]:
# Conversão dos tipos numéricos e categóricos
merged_dataset = merged_dataset.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int', 
    'DIAS_ATRASO': 'Int64',
    'TARGET_INADIMPLENCIA': 'int',
    'SEGMENTO_INDUSTRIAL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
    'DIAS_ADIANTAMENTO': 'int',
    'DOMINIO_EMAIL': 'category',
})

# Conversão de datas
merged_dataset['SAFRA_REF'] = pd.to_datetime(
    merged_dataset['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    merged_dataset['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_PAGAMENTO'] = pd.to_datetime(
    merged_dataset['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_VENCIMENTO'] = pd.to_datetime(
    merged_dataset['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_CADASTRO'] = pd.to_datetime(
    merged_dataset['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


Criando novas features (pense nelas)

Calculando tempo de casa. Indica a relação dos clientes com a empresa, isso pode influenciar na inadimplência. Clientes mais velhos de casa têm menos tendência a inadimplência.

In [125]:
merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_CADASTRO']).dt.days

In [126]:
merged_dataset['TEMPO_DE_CASA_MESES'] = merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] // 30


In [127]:
merged_dataset = merged_dataset.drop(columns='TEMPO_CADASTRO_PARA_VENCIMENTO')


Prazo para o pagamento: ajuda a entender o tempo que o cliente teve para se preparar para o pagamento. Prazos curtos podem influenciar na inadimplência.



In [128]:
merged_dataset['PRAZO_PAGAMENTO_DIAS'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_EMISSAO_DOCUMENTO']).dt.days

Mês/Ano da SAFRA_REF: capta padrões sazonais (ex: pode haver mais inadimplência no final do ano ou em meses com impostos).

In [129]:
merged_dataset['MES_SAFRA'] = merged_dataset['SAFRA_REF'].dt.month
merged_dataset['ANO_SAFRA'] = merged_dataset['SAFRA_REF'].dt.year


Criação de **INADIMPLENCIAS_ANTERIORES**. Essa variável é um contador cumulativo do número de inadimplências passadas para cada cliente até o momento do pagamento atual. Ela captura o comportamento passado do cliente, que é um forte indicador de risco de inadimplência. Clientes reincidentes tendem a ter maior probabilidade de inadimplir novamente.

In [130]:
# Ordena por cliente e data de emissão
merged_dataset = merged_dataset.sort_values(by=['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO'])

# Cria coluna cumulativa de inadimplências
merged_dataset['INADIMPLENCIAS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['TARGET_INADIMPLENCIA']
    .cumsum()
    .shift(fill_value=0)  # Para não contar a linha atual
)


In [131]:
merged_dataset.dtypes

ID_CLIENTE                            int64
SAFRA_REF                    datetime64[ns]
DATA_EMISSAO_DOCUMENTO       datetime64[ns]
DATA_PAGAMENTO               datetime64[ns]
DATA_VENCIMENTO              datetime64[ns]
VALOR_A_PAGAR                       float64
TAXA                                float64
RENDA_MES_ANTERIOR                  float64
NO_FUNCIONARIOS                       int64
DATA_CADASTRO                datetime64[ns]
FLAG_PF                               int64
SEGMENTO_INDUSTRIAL                category
DOMINIO_EMAIL                      category
PORTE                              category
CEP_2_DIG                          category
DIAS_ATRASO                           Int64
TARGET_INADIMPLENCIA                  int64
DIAS_ADIANTAMENTO                     int64
TEMPO_DE_CASA_MESES                   int64
PRAZO_PAGAMENTO_DIAS                  int64
MES_SAFRA                             int32
ANO_SAFRA                             int32
INADIMPLENCIAS_ANTERIORES       

In [132]:
merged_dataset[['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'SAFRA_REF', 'RENDA_MES_ANTERIOR']]

,ID_CLIENTE,DATA_EMISSAO_DOCUMENTO,SAFRA_REF,RENDA_MES_ANTERIOR
0,8784237149961904,2018-09-04,2018-09-01,300502.0
1,8784237149961904,2018-09-06,2018-09-01,300502.0
2,8784237149961904,2018-09-09,2018-09-01,300502.0
3,8784237149961904,2018-09-11,2018-09-01,300502.0
4,8784237149961904,2018-09-17,2018-09-01,300502.0
...,...,...,...,...
72485,9206030810342980458,2021-05-16,2021-05-01,256133.0
72486,9206030810342980458,2021-05-23,2021-05-01,256133.0
72487,9206030810342980458,2021-06-16,2021-06-01,463963.0
72488,9206030810342980458,2021-06-18,2021-06-01,463963.0


Pré-processamento dos dados para o modelo 

Colunas a serem removidas:

- ID_CLIENTE: funciona como um identificador, não possui informação útil para predição.
- DATA_EMISSAO_DOCUMENTO, DATA_PAGAMENTO, DATA_VENCIMENTO, DATA_CADASTRO, SAFRA_REF: São datas brutas. Foram transformadas em variáveis derivadas.


Colunas categóricas a serem tratadas: 
- SEGMENTO_INDUSTRIAL, PORTE, CEP_2_DIG, DOMINIO_EMAIL: aplicar One-Hot Encoding 



# LIDAR COM A ALTA CARDINALIDADE DE CEP_2_DIG POSTERIORMENTE


Aplicando o pré-processamento 

In [133]:
# Eliminar colunas irrelevantes
cols_drop = [
    'ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
    'DATA_VENCIMENTO', 'DATA_CADASTRO', 'SAFRA_REF', 'CEP_2_DIG',
    'DIAS_ATRASO', 'DIAS_ADIANTAMENTO' # Colunas que só são conhecidas depois do treinamento
]

df = merged_dataset.drop(columns=cols_drop)


# Aplicando One-Hot Encoding em variáveis categóricas de baixa cardinalidade
df = pd.get_dummies(df, columns=['SEGMENTO_INDUSTRIAL', 'PORTE', 'DOMINIO_EMAIL'], drop_first=True)

X = df.drop(columns='TARGET_INADIMPLENCIA')
y = df['TARGET_INADIMPLENCIA']

In [134]:
X

,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,FLAG_PF,TEMPO_DE_CASA_MESES,PRAZO_PAGAMENTO_DIAS,MES_SAFRA,ANO_SAFRA,INADIMPLENCIAS_ANTERIORES,SEGMENTO_INDUSTRIAL_Indústria,SEGMENTO_INDUSTRIAL_Serviços,PORTE_MEDIO,PORTE_PEQUENO,DOMINIO_EMAIL_HOTMAIL,DOMINIO_EMAIL_OUTROS,DOMINIO_EMAIL_YAHOO
0,59610.76,5.99,300502.0,107,0,92,20,9,2018,0,False,False,False,True,True,False,False
1,39398.06,5.99,300502.0,107,0,92,18,9,2018,0,False,False,False,True,True,False,False
2,55416.75,5.99,300502.0,107,0,92,16,9,2018,0,False,False,False,True,True,False,False
3,11751.35,5.99,300502.0,107,0,92,16,9,2018,0,False,False,False,True,True,False,False
4,35985.00,5.99,300502.0,107,0,92,16,9,2018,0,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72485,107321.36,11.99,256133.0,109,0,192,16,5,2021,2,False,True,True,False,True,False,False
72486,38372.91,5.99,256133.0,109,0,192,16,5,2021,2,False,True,True,False,True,False,False
72487,43030.50,5.99,463963.0,105,0,193,16,6,2021,2,False,True,True,False,True,False,False
72488,107318.81,5.99,463963.0,105,0,193,17,6,2021,2,False,True,True,False,True,False,False


In [135]:
y

0        0
1        0
2        0
3        0
4        0
        ..
72485    0
72486    0
72487    0
72488    0
72489    0
Name: TARGET_INADIMPLENCIA, Length: 72490, dtype: int64

In [138]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# === 1. PREPARAÇÃO DOS DADOS ===
# Supondo que você já tenha seu dataframe pronto com as variáveis transformadas
# e a variável target separada como `TARGET_INADIMPLENCIA`

# Exemplo:
X = df.drop(columns=['TARGET_INADIMPLENCIA'])
y = df['TARGET_INADIMPLENCIA']

# Divisão treino/teste (estratificada por causa do desbalanceamento)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === 2. DEFINIÇÃO DO MODELO BASE ===
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),  # Contornando o desbalanceamento
)

# === 3. GRID DE PARÂMETROS ===
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# === 4. VALIDAÇÃO CRUZADA ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=3
)

# === 5. TREINAMENTO ===
grid.fit(X_train, y_train)

# === 6. RESULTADOS ===
print("Melhores hiperparâmetros:")
print(grid.best_params_)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_proba))


Fitting 5 folds for each of 72 candidates, totalling 360 fits


c:\Users\Matheus\Documents\GitHub\default-risk-analysis\dra-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:00:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Melhores hiperparâmetros:
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     13432
           1       0.55      0.83      0.66      1066

    accuracy                           0.94     14498
   macro avg       0.77      0.89      0.81     14498
weighted avg       0.95      0.94      0.94     14498


Confusion Matrix:
[[12696   736]
 [  179   887]]

ROC AUC Score:
0.9562053654737308


In [137]:

#merged_dataset.to_csv('../data/processed/train_merged_dataset.csv', index=False)